# Leveraging the Jupyter and IPython display protocol

This is a small essay to show how one can make a better use of the display protocol. All you will see in this blog post has been available for a couple of years but no one really built on top of this. 

It is usually known that the IPython rich display mechanism allows library authors to define rich representation for 
their objects. You may have seen it in [SymPy](http://www.sympy.org/en/index.html), which makes extensive use of the latex representation, and [Pandas](http://pandas.pydata.org/) which offers a nice HTML view of dataframes. 

What I'm going to show below, is that one is not limited to these – you can alter the representation of any existing object without modifying its source – and that this can be used to alter the view of containers, with the example of lists, to make things easy to read. 

## Modifying objects reprs

This section is just a reminder of how one can change defined representation for objects whose source code is under your 
control. When defining a class, the code author needs to define a number of methods which should return the `(data, metadata)` pair for a given object mimetype. If no metadata is necessary, these can be omitted. For some common representations, short method names are available. These methods can be recognized as they all follow the following pattern: `_repr_*_(self)`. That is to say, an underscore, followed by `repr` followed by an underscore. The star `*` needs to be replaced by a lowercase identifier often referring to a short human readable description of the format (e.g.: `png` , `html`, `pretty`, ...). Finish with a single underscore. We note that unlike the python `__repr__` (pronounced "Dunder rep-er" which starts and ends with two underscores, the "Rich reprs" or "Reprs-stars" start and end with a _single_ underscore.

Here is the class definition of a simple object that implements three of the rich representation methods:

  - "text/html" via the `_repr_html_` method
  - "text/latex" via the `_repr_latex_` method
  - "text/markdown" via the `_repr_markdown_` method

None of these methods returns a tuple, thus IPython will infer that there is no associated metadata.
  
The "text/plain" mimetype representation is provided by the classical Python's `__repr__(self)`.

In [ ]:
print("remember to clear all outputs :-) ")

In [6]:
class MultiMime:
    
    def __repr__(self):
        return "this is the repr"
    
    def _repr_html_(self):
        return "This <b>is</b> html"
    
    def _repr_markdown_(self):
        return "This **is** markdown"

    def _repr_latex_(self):
        return "$ Latex \otimes mimetype $"

In [7]:
display(MultiMime())

this is the repr

In [8]:
MultiMime()

this is the repr

In [ ]:
# restart the kernel

In [ ]:
MultiMime()

All the mimetype representations will be sent to the front end (in many cases the notebook web interface), and the richer one will be picked and displayed to the user. All representations are stored in the notebook document (on disk) and this can be chosen from when the document is later reopened – even with no kernel attached – or converted to another format. 

## External formatters and containers

As stated in the introduction, you do not need to have control over an object source code to change its representation. Still it is often a more convenient process. As an example, we will build a Container for image thumbnails and see how we can use the code written for this custom container to apply it to generic Python containers like lists. 

As a visual example we'll use Orly Parody books covers, in particular a small resolution of some of them to limit the amount of data we'll be working with. 

In [3]:
cd thumb

/Users/bussonniermatthias/dev/orly-full-res/thumb


Let's see some of the images present in this folder:

In [4]:
names = !ls *.png
names[:20], f"{len(names) - 10} more"

In the above I've used an IPython-specific syntax (`!ls`) to conveniently extract all the files with a png extension (`*.png`) in the current working directory, and assign this to the `names` variable.

That's cute, but not really useful for images. We know we can display images in the Jupyter notebook when using the IPython kernel, for that we can use the `Image` class situated in the `IPython.display` submodule. We can construct such an object simply by passing the filename. `Image` already provides a rich representation:

In [5]:
from IPython.display import Image

In [ ]:
im = Image(names[0])
im

The raw data from the image file is available via the `.data` attribute:

In [ ]:
im.data[:20]

What if we map `Images` to each element of a list ?

In [ ]:
from random import choices
mylist = list(map(Image, set(choices(names, k=10))))
mylist

Well unfortunately a list object only knows how to represent itself using text and the text representation of its elements. We'll have to build a thumbnail gallery ourself.

First let's (re)-build an HTML representation for display a single image:

In [ ]:
import base64
from IPython.display import HTML
def tag_from_data(data, size='100%'):
    return (
        '''<img
             style="display:inline;
                    width:{1};
                    max-width:400px;
                    padding:10px;
                    margin-top:14px"
             src="data:image/png;base64,{0}"
           />
           ''').format(''.join(base64.encodebytes(data).decode().split('\n')), size)

We encode the data from bytes to base64 (newline separated), and strip the newlines. We format that into an Html template – with some inline style – and set the source (`src` to be this base64 encoded string). We can check that this display correctly by wrapping the all thing in an `HTML` object that provide a conveninent `_repr_html_`. 

In [ ]:
HTML(tag_from_data(im.data))

Now we can create our own subclass, hich take a list of images and contruct and HTML representation for each of these, then join them together. We define and define a `_repr_html_`, that wrap the all in a paragraph tag, and add a comma between each image: 

In [ ]:
class VignetteList:
    
    
    def __init__(self, *images, size=None):
        self.images = images
        self.size = size
        
    def _repr_html_(self):
        return '<p>'+','.join(tag_from_data(im.data, self.size)  for im in self.images)+'</p>'
    
    def _repr_latex_(self):
        return '$ O^{rly}_{books} (%s\ images)$ ' % (len(self.images))
        

We also define a LaTeX Representation – that we will not use here, and look at our newly created object using previously defined list:

In [ ]:
VignetteList(*mylist, size='200px')

That is nice, though it forces us to unpack all the lists we have explicitely into a VignetteList – which may be annoying. Let's cleanup a bit the above, and register an external formatter for the "text/html" mimetype that should be used for any object which is a `list`. We'll also improve the formatter to recusrse in objects. THat is to say: 
  - If it's an image return the PNG data in an `<img>` tag, 
  - If it's an object that has an `text/html` reprensetation, use that. 
  - Otherwise, use th repr. 
  
With this we loose some nice formatting of text lists with the pretty module, we could easily fix that; but we leve it as an exercice for the reader. 

In [ ]:
def tag_from_data_II(data, size='100%'):
    return '''<img
                    style="
                        display:inline;
                        width:{1};
                        max-width:400px;
                        padding:10px;
                        margin-top:14px"
                    onMouseOver="this.style['box-shadow']='5px 5px 30px 0px rgba(163,163,163,1)'" 
                    onMouseOut="this.style['box-shadow']=''"
                    src="data:image/png;base64,{0}" 
             />'''.format(''.join(base64.encodebytes(data).decode().split('\n')), size)

def html_list_formatter(ll):
    html = get_ipython().display_formatter.formatters['text/html']
    reps = []
    for o in ll:
        if isinstance(o, Image):
            reps.append(tag_from_data_II(o.data, '200px') )
        else: 
            h = html(o)
            if h:    
                reps.append(h)
            else:
                reps.append(repr(o)+'')
    
    return '<span>['+','.join(reps)+']</span>'

Same as before, with square bracket after and before, and a bit of styling that change the drop shadow on hover. Now we register the above with IPython:

In [ ]:
mylist

In [ ]:
ipython = get_ipython()
html_formatter = ipython.display_formatter.formatters['text/html']
html_formatter.for_type(list, html_list_formatter)

In [ ]:
mylist

## Disp

External integration for some already existing object is available in [disp](https://github.com/ipython/disp), in particular you will find representation for SparkContext, `requests`'s `Responses` object (collapsible json content and headers), as well as a couple others. 

In [ ]:
from requests.models import Response
import requests

In [ ]:
def req_formatter(response):
    return "&lt;This is a response object&gt;"

In [ ]:
html_formatter.for_type(Response, req_formatter)

In [ ]:
r = requests.get('https://api.github.com')
r

### Existing formatters

In [ ]:
from disp.py3only import html_formatter_for_Response
html_formatter.for_type(Response, html_formatter_for_Response)

In [ ]:
r

### Completion

static inference is a possibility when the code has not been executed:

In [ ]:
l = ["a", 1, html_formatter]
l[0].capitalize

Some information are hard to infer, or cannot be infered:

In [ ]:
from random import choice
el = choice(l)

For this the kernel may decide to actually inspect the current underlying state:

In [ ]:
el.

## Magic integration, cross languages

The above demonstatratino show that a kernel is more than a language, it is a controling process that manage user requests (in our case code execution) and how the results are returned to the user. There is often the assumtion that a kernel is a single language, this is an incorrect assumtion as a kernl proces may manage several language and can orchestrate data movement from one language to another. 

In the following we can see how a Python process make use of what we have defined above to make sql querries returning rich results. We also see that the execution od SQL queries have side effects in the Python namespace, showing how the kernel can orchestrate things. 

In [ ]:
load_ext fakesql

In [ ]:
try:
    rly
except NameError:
    print('`rly` not defined')

In [ ]:
%%sql
SELECT name,cover from orly WHERE color='purple' LIMIT 5

In [ ]:
rly[2]

It would not be hard to have modification of the Python namespace to affect the SQL database, this is left as an exercise to the user as well (hint use properties) and to have integration with other languages like R, Julia, ... 

## More languages:

See [here](http://carreau.github.io/posts/23-Cross-Language-Integration.html) for R,C, Python, Cython, Rust, Fortran, Julia in the same notebook.

Note:

> This notebook has initially been written to display prototype features of IPython and the Jupyter notebook, in particular completions of cell magic (for the Sql Cell), and UI element allowing to switch between the shown mimetype. This will not be reflected in static rendering and is not mentioned in the text, which may lead to a confusing read. 